In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

Статистику поступили данные пилотного исследования, в котором анализировались две группы пациентов (группы A1 и A2, n1, n2 наблюдений в каждой группе соответственно) с разными подходами лечения.
Разница успешности лечения измеряется при помощи двух показателей эффективности (непрерывные, нормально распределенные).
Необходимо проверить, значимо ли отличаются средние значения каждого показателя эффективности между группами.
Статистик принимает решение о значимости всего эксперимента на уровне alpha = 0,025.
Каковы наиболее консервативные и достаточные требования, которые можно наложить на проводимые тесты, чтобы признать результаты значимыми для заданного alpha?

Перед нами задача множественной проверки гипотез. Самый простой способ решить поставленную выше задачу — это использовать поправку Бонферрони. В
методе Бонферрони достигаемые уровни значимости всех проверяемых гипотез сравниваются с величиной $\frac{\alpha}{m}$, в данном случае  $m = 2 $

Для двух групп врачей: ортодонтов и стоматологов имеются данные об использовании типа оборудования для снятия слепков зубов (аналоговое vs цифровое). Из 200 ортодонтов 130 используют цифровое оборудование для 230 стоматологов эта цифра ранва 102. Можно ли сказать, что тип существует зависиомсть между двумя факторами: группа врачей и тип оборудования  (ошибка первого рода равна 5%)?

## Доверительный интервал для разности долей (независимые выборки)

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [14]:
orthodontists =[0]*70+[1]*130
dentists =[0]*128+[1]*102

In [15]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [16]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [17]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)



In [18]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(orthodontists, dentists)

95% confidence interval for a difference between proportions: [0.114371, 0.298673]


In [19]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(orthodontists, dentists),'two-sided')

p-value: 0.000018


Гипотеза о равенстве долей использующих цифоровое оборудование среди ортодонтов и стоматологов отвергается в пользу двухсторонней альтернативы. 